<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/TrainTest_400.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import PIL
from PIL import Image

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore

#### Load data

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class/Image'

train 3 class

In [16]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

#### train 3 class  --- สร้างตารางเก็บ ชื่อภาพ และชื่อคลาส
ข้อมูลใน All_3class_dir รวมภาพทั้ง 3 คลาส รวม 451 ภาพ TB150, Normal151 และ CA150 (CA คือ lungcancer)

In [17]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

In [18]:
normal = []
for i in range(len(os.listdir(Normal151_dir))):
    img = os.listdir(Normal151_dir)[i]
    normal.append(img)
    
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [19]:
tuberculosis = []
for i in range(len(os.listdir(TB150_dir))):
    img = os.listdir(TB150_dir)[i]
    tuberculosis.append(img)

ttuberculosis = pd.DataFrame({'ImageName':[tuberculosis][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [20]:
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [21]:
data_train3 = pd.concat([tnormal, ttuberculosis, tlungcancer])
data_train3[-3:]

,ImageName,NameType,NumberType
147,ca2.jpg,Lungcancer,2
148,ca26.jpg,Lungcancer,2
149,ca8.jpg,Lungcancer,2


#  ( Crop ) Train Test 0-400

##1 softmax   --> Crop  
 
 0.73 // 28min

*   BS = 1
*   fix weight & bias in layer 0-400
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [22]:
train_img_Dense = []
mywidth = 224 
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป

    img = Image.open(image_path).convert('RGB')     

    wpercent = (mywidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
    img = img.crop((0,0,mywidth,mywidth))

    data_Dense = np.array(img, dtype="float32") / 255.0
    train_img_Dense.append(data_Dense)

data_Dense = np.array(train_img_Dense)

100%|██████████| 451/451 [02:18<00:00,  3.25it/s]


In [23]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [24]:
#ใช้ภาพที่ครอบแล้ว ใน data_Dense2
(x_train_Dense2, x_test_Dense2, y_train_Dense2, y_test_Dense2 ) = train_test_split(data_Dense, labels_Dense,
	                                                                                                test_size=0.25, random_state=40)

In [25]:
le_Dense.classes_                  # มีค่า ['Lungcancer', 'Normal', 'Tuberculosis']

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [26]:
INIT_LR = 1e-1
BS = 1
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [27]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 1s 0us/step


707

In [28]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:400]:    #fix w & bias in layer 0-400
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [29]:
H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS,
	                                            epochs=30,  callbacks = [callback])

Epoch 1/30
338/338 [==============================] - 253s 674ms/step - loss: 0.6366 - accuracy: 0.3580 - val_loss: 0.5062 - val_accuracy: 0.5929
Epoch 2/30
338/338 [==============================] - 373s 1s/step - loss: 0.5415 - accuracy: 0.5680 - val_loss: 0.4531 - val_accuracy: 0.6549
Epoch 3/30
338/338 [==============================] - 366s 1s/step - loss: 0.4430 - accuracy: 0.6568 - val_loss: 0.7134 - val_accuracy: 0.5398
Epoch 4/30
338/338 [==============================] - 349s 1s/step - loss: 0.3880 - accuracy: 0.7189 - val_loss: 0.4405 - val_accuracy: 0.6903
Epoch 5/30
338/338 [==============================] - 364s 1s/step - loss: 0.3729 - accuracy: 0.7308 - val_loss: 0.6663 - val_accuracy: 0.6726
Epoch 6/30
338/338 [==============================] - 369s 1s/step - loss: 0.3188 - accuracy: 0.7988 - val_loss: 0.4669 - val_accuracy: 0.7257
Epoch 7/30
338/338 [==============================] - 366s 1s/step - loss: 0.2877 - accuracy: 0.8047 - val_loss: 0.6188 - val_accuracy: 0.6

In [30]:
#1
pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_Dense2.argmax(axis=1),  
                                    target_names=le_Dense.classes_))

              precision    recall  f1-score   support

  Lungcancer       1.00      0.23      0.37        35
      Normal       0.83      0.93      0.87        41
Tuberculosis       0.56      0.89      0.69        37

    accuracy                           0.70       113
   macro avg       0.80      0.68      0.64       113
weighted avg       0.79      0.70      0.66       113



In [31]:
score = metrics.accuracy_score(y_test_Dense2.argmax(axis=1),	        #ค่า acc
                                             pred_Dense2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

Fold score (accuracy): 0.6991150442477876


In [32]:
pred = np.round(pred_Dense2, 3)

In [33]:
le_Dense.classes_                  # มีค่า ['Lungcancer', 'Normal', 'Tuberculosis']

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

##1 sigmoid   --> Crop  

0.73 // 26 min

*   BS = 1
*   fix weight & bias in layer 0-400
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [34]:
train_img_Dense = []
mywidth = 224 
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป

    img = Image.open(image_path).convert('RGB')     

    wpercent = (mywidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
    img = img.crop((0,0,mywidth,mywidth))

    data_Dense = np.array(img, dtype="float32") / 255.0
    train_img_Dense.append(data_Dense)

data_Dense = np.array(train_img_Dense)

100%|██████████| 451/451 [00:08<00:00, 55.33it/s]


In [35]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [36]:
(x_train_Dense2, x_test_Dense2, y_train_Dense2, y_test_Dense2 ) = train_test_split(data_Dense, labels_Dense,
	                                                                                                test_size=0.25, random_state=40)

In [37]:
le_Dense.classes_                  # มีค่า ['Lungcancer', 'Normal', 'Tuberculosis']

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [38]:
INIT_LR = 1e-1
BS = 1
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [39]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [40]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:400]:    #fix w & bias in layer 0-400
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [41]:
H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS,
	                                            epochs=30,  callbacks = [callback])

Epoch 1/30
338/338 [==============================] - 280s 754ms/step - loss: 0.6284 - accuracy: 0.4112 - val_loss: 0.5499 - val_accuracy: 0.5929
Epoch 2/30
338/338 [==============================] - 375s 1s/step - loss: 0.5152 - accuracy: 0.5888 - val_loss: 0.4054 - val_accuracy: 0.6814
Epoch 3/30
338/338 [==============================] - 397s 1s/step - loss: 0.4215 - accuracy: 0.6805 - val_loss: 0.3740 - val_accuracy: 0.7257
Epoch 4/30
338/338 [==============================] - 387s 1s/step - loss: 0.3928 - accuracy: 0.6923 - val_loss: 0.3991 - val_accuracy: 0.7080
Epoch 5/30
338/338 [==============================] - 394s 1s/step - loss: 0.3598 - accuracy: 0.7515 - val_loss: 0.5327 - val_accuracy: 0.6814
Epoch 6/30
338/338 [==============================] - 383s 1s/step - loss: 0.3243 - accuracy: 0.7959 - val_loss: 0.4139 - val_accuracy: 0.7611
Epoch 7/30
338/338 [==============================] - 378s 1s/step - loss: 0.2828 - accuracy: 0.8284 - val_loss: 0.4492 - val_accuracy: 0.7

In [42]:
#1
pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_Dense2.argmax(axis=1),  
                                    target_names=le_Dense.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.60      0.51      0.55        35
      Normal       0.93      0.90      0.91        41
Tuberculosis       0.53      0.62      0.57        37

    accuracy                           0.69       113
   macro avg       0.69      0.68      0.68       113
weighted avg       0.70      0.69      0.69       113



In [43]:
score = metrics.accuracy_score(y_test_Dense2.argmax(axis=1),	        #ค่า acc
                                             pred_Dense2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

Fold score (accuracy): 0.6902654867256637


In [44]:
pred = np.round(pred_Dense2, 3)

In [45]:
le_Dense.classes_                  # มีค่า ['Lungcancer', 'Normal', 'Tuberculosis']

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

##8 softmax   --> Crop

0.72 // 18 min

*   BS = 8
*   fix weight & bias in layer 0-400
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
mywidth = 224 
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป

    img = Image.open(image_path).convert('RGB')     

    wpercent = (mywidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
    img = img.crop((0,0,mywidth,mywidth))

    data_Dense = np.array(img, dtype="float32") / 255.0
    train_img_Dense.append(data_Dense)

data_Dense = np.array(train_img_Dense)

100%|██████████| 451/451 [00:07<00:00, 59.89it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
(x_train_Dense2, x_test_Dense2, y_train_Dense2, y_test_Dense2 ) = train_test_split(data_Dense, labels_Dense,
	                                                                                                test_size=0.25, random_state=40)

In [ ]:
le_Dense.classes_                  # มีค่า ['Lungcancer', 'Normal', 'Tuberculosis']

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [ ]:
INIT_LR = 1e-1
BS8 = 8
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:400]:    #fix w & bias in layer 0-400
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS8),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS8,
	                                            epochs=30,  callbacks = [callback])

Epoch 1/30
42/42 [==============================] - 174s 4s/step - loss: 0.4848 - accuracy: 0.6182 - val_loss: 0.6124 - val_accuracy: 0.4425
Epoch 2/30
42/42 [==============================] - 149s 4s/step - loss: 0.3739 - accuracy: 0.7515 - val_loss: 0.5205 - val_accuracy: 0.5752
Epoch 3/30
42/42 [==============================] - 146s 3s/step - loss: 0.2879 - accuracy: 0.8242 - val_loss: 0.4019 - val_accuracy: 0.7522
Epoch 4/30
42/42 [==============================] - 145s 3s/step - loss: 0.2454 - accuracy: 0.8667 - val_loss: 0.5134 - val_accuracy: 0.7080
Epoch 5/30
42/42 [==============================] - 149s 4s/step - loss: 0.2295 - accuracy: 0.8576 - val_loss: 0.5668 - val_accuracy: 0.7080
Epoch 6/30
42/42 [==============================] - 149s 4s/step - loss: 0.2140 - accuracy: 0.8879 - val_loss: 0.6900 - val_accuracy: 0.7168


In [ ]:
#1
pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_Dense2.argmax(axis=1),  
                                    target_names=le_Dense.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.56      0.91      0.70        35
      Normal       0.87      0.98      0.92        41
Tuberculosis       0.90      0.24      0.38        37

    accuracy                           0.72       113
   macro avg       0.78      0.71      0.67       113
weighted avg       0.78      0.72      0.67       113



In [ ]:
score = metrics.accuracy_score(y_test_Dense2.argmax(axis=1),	        #ค่า acc
                                             pred_Dense2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

Fold score (accuracy): 0.7168141592920354


In [ ]:
pred = np.round(pred_Dense2, 3)

##8 sigmoid   --> Crop

0.73

*   BS = 8
*   fix weight & bias in layer 0-400
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
mywidth = 224 
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป

    img = Image.open(image_path).convert('RGB')     

    wpercent = (mywidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
    img = img.crop((0,0,mywidth,mywidth))

    data_Dense = np.array(img, dtype="float32") / 255.0
    train_img_Dense.append(data_Dense)

data_Dense = np.array(train_img_Dense)

100%|██████████| 451/451 [00:07<00:00, 60.81it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
(x_train_Dense2, x_test_Dense2, y_train_Dense2, y_test_Dense2 ) = train_test_split(data_Dense, labels_Dense,
	                                                                                                test_size=0.25, random_state=40)

In [ ]:
le_Dense.classes_                  # มีค่า ['Lungcancer', 'Normal', 'Tuberculosis']

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [ ]:
INIT_LR = 1e-1
BS8 = 8
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:400]:    #fix w & bias in layer 0-400
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS8),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS8,
	                                            epochs=30,  callbacks = [callback])

Epoch 1/30
42/42 [==============================] - 172s 4s/step - loss: 0.5405 - accuracy: 0.5982 - val_loss: 0.5983 - val_accuracy: 0.4956
Epoch 2/30
42/42 [==============================] - 145s 3s/step - loss: 0.3369 - accuracy: 0.7636 - val_loss: 0.5068 - val_accuracy: 0.6460
Epoch 3/30
42/42 [==============================] - 150s 4s/step - loss: 0.2905 - accuracy: 0.8455 - val_loss: 0.5202 - val_accuracy: 0.6814
Epoch 4/30
42/42 [==============================] - 151s 4s/step - loss: 0.2322 - accuracy: 0.8727 - val_loss: 0.3630 - val_accuracy: 0.7876
Epoch 5/30
42/42 [==============================] - 151s 4s/step - loss: 0.2267 - accuracy: 0.8758 - val_loss: 0.3917 - val_accuracy: 0.7522
Epoch 6/30
42/42 [==============================] - 151s 4s/step - loss: 0.1817 - accuracy: 0.8939 - val_loss: 0.3479 - val_accuracy: 0.8053
Epoch 7/30
42/42 [==============================] - 151s 4s/step - loss: 0.2066 - accuracy: 0.9000 - val_loss: 0.4976 - val_accuracy: 0.7522
Epoch 8/30
42

In [ ]:
#1
pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_Dense2.argmax(axis=1),  
                                    target_names=le_Dense.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.55      0.89      0.68        35
      Normal       0.97      0.90      0.94        41
Tuberculosis       0.79      0.41      0.54        37

    accuracy                           0.73       113
   macro avg       0.77      0.73      0.72       113
weighted avg       0.78      0.73      0.73       113



In [ ]:
score = metrics.accuracy_score(y_test_Dense2.argmax(axis=1),	        #ค่า acc
                                             pred_Dense2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

Fold score (accuracy): 0.7345132743362832


In [ ]:
pred = np.round(pred_Dense2, 3)

##16 softmax  --> Crop

0.68 // 19 min

*   BS = 16
*   fix weight & bias in layer 0-400
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
mywidth = 224 
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป

    img = Image.open(image_path).convert('RGB')     

    wpercent = (mywidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
    img = img.crop((0,0,mywidth,mywidth))

    data_Dense = np.array(img, dtype="float32") / 255.0
    train_img_Dense.append(data_Dense)

data_Dense = np.array(train_img_Dense)

100%|██████████| 451/451 [02:13<00:00,  3.37it/s]


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
(x_train_Dense2, x_test_Dense2, y_train_Dense2, y_test_Dense2 ) = train_test_split(data_Dense, labels_Dense,
	                                                                                                test_size=0.25, random_state=40)

In [ ]:
le_Dense.classes_                  # มีค่า ['Lungcancer', 'Normal', 'Tuberculosis']

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [ ]:
INIT_LR = 1e-1
BS16 = 16
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

74850304/74836368 [==============================] - 1s 0us/step


707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:400]:    #fix w & bias in layer 0-400
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS16),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS16,
	                                            epochs=30,  callbacks = [callback])

Epoch 1/30
21/21 [==============================] - 198s 8s/step - loss: 0.5133 - accuracy: 0.5901 - val_loss: 0.4926 - val_accuracy: 0.6726
Epoch 2/30
21/21 [==============================] - 176s 8s/step - loss: 0.3538 - accuracy: 0.7946 - val_loss: 0.4817 - val_accuracy: 0.6549
Epoch 3/30
21/21 [==============================] - 171s 8s/step - loss: 0.2866 - accuracy: 0.8199 - val_loss: 0.5297 - val_accuracy: 0.6195
Epoch 4/30
21/21 [==============================] - 172s 8s/step - loss: 0.2423 - accuracy: 0.8820 - val_loss: 0.5214 - val_accuracy: 0.6726


In [ ]:
#1
pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_Dense2.argmax(axis=1),  
                                    target_names=le_Dense.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.62      0.74      0.68        35
      Normal       0.67      0.98      0.79        41
Tuberculosis       0.91      0.27      0.42        37

    accuracy                           0.67       113
   macro avg       0.73      0.66      0.63       113
weighted avg       0.73      0.67      0.63       113



In [ ]:
score = metrics.accuracy_score(y_test_Dense2.argmax(axis=1),	        #ค่า acc
                                             pred_Dense2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

Fold score (accuracy): 0.672566371681416


In [ ]:
pred = np.round(pred_Dense2, 2)

In [ ]:
le_Dense.classes_

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [ ]:
pred[:6]

array([[0.83, 0.16, 0.01],
       [0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [0.34, 0.6 , 0.06],
       [0.99, 0.  , 0.01]], dtype=float32)

In [ ]:
y_test_Dense2[:6]

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

##16 sigmoid  --> Crop

0.73 // 22min

*   BS = 16
*   fix weight & bias in layer 0-400
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
mywidth = 224 
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป

    img = Image.open(image_path).convert('RGB')     

    wpercent = (mywidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
    img = img.crop((0,0,mywidth,mywidth))

    data_Dense = np.array(img, dtype="float32") / 255.0
    train_img_Dense.append(data_Dense)

data_Dense = np.array(train_img_Dense)

100%|██████████| 451/451 [00:07<00:00, 61.14it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
(x_train_Dense2, x_test_Dense2, y_train_Dense2, y_test_Dense2 ) = train_test_split(data_Dense, labels_Dense,
	                                                                                                test_size=0.25, random_state=40)

In [ ]:
le_Dense.classes_                  # มีค่า ['Lungcancer', 'Normal', 'Tuberculosis']

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [ ]:
INIT_LR = 1e-1
BS16 = 16
EPOCHS = 50

aug51 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:400]:    #fix w & bias in layer 0-400
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense51 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense51.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
H_Dense51 = model_Dense51.fit( x=aug51.flow(x_train_Dense2, y_train_Dense2, batch_size=BS16),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS16,
	                                            epochs=30,  callbacks = [callback])

Epoch 1/30
21/21 [==============================] - 167s 7s/step - loss: 0.4945 - accuracy: 0.6398 - val_loss: 0.6170 - val_accuracy: 0.4690
Epoch 2/30
21/21 [==============================] - 141s 7s/step - loss: 0.2995 - accuracy: 0.8323 - val_loss: 0.6350 - val_accuracy: 0.4956
Epoch 3/30
21/21 [==============================] - 141s 7s/step - loss: 0.2574 - accuracy: 0.8416 - val_loss: 0.6600 - val_accuracy: 0.5310
Epoch 4/30
21/21 [==============================] - 146s 7s/step - loss: 0.2176 - accuracy: 0.8899 - val_loss: 0.8356 - val_accuracy: 0.5133
Epoch 5/30
21/21 [==============================] - 139s 7s/step - loss: 0.1502 - accuracy: 0.9286 - val_loss: 0.5210 - val_accuracy: 0.6549
Epoch 6/30
21/21 [==============================] - 139s 7s/step - loss: 0.1406 - accuracy: 0.9317 - val_loss: 0.3903 - val_accuracy: 0.7611
Epoch 7/30
21/21 [==============================] - 138s 7s/step - loss: 0.0972 - accuracy: 0.9565 - val_loss: 0.4705 - val_accuracy: 0.7080
Epoch 8/30
21

In [ ]:
pred_Dense2 = model_Dense51.predict(x_test_Dense2)
    
print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_Dense2.argmax(axis=1),  
                                    target_names=le_Dense.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.60      0.74      0.67        35
      Normal       0.87      0.95      0.91        41
Tuberculosis       0.68      0.46      0.55        37

    accuracy                           0.73       113
   macro avg       0.72      0.72      0.71       113
weighted avg       0.72      0.73      0.72       113



In [ ]:
score = metrics.accuracy_score(y_test_Dense2.argmax(axis=1),	        #ค่า acc
                                             pred_Dense2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

Fold score (accuracy): 0.7256637168141593


In [ ]:
pred = np.round(pred_Dense2, 3)

##32 softmax   --> Crop

0.67 // 14 min

*   BS = 32
*   fix weight & bias in layer 0-400
*   activation="softmax"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
mywidth = 224 
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป

    img = Image.open(image_path).convert('RGB')     

    wpercent = (mywidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
    img = img.crop((0,0,mywidth,mywidth))

    data_Dense = np.array(img, dtype="float32") / 255.0
    train_img_Dense.append(data_Dense)

data_Dense = np.array(train_img_Dense)

100%|██████████| 451/451 [00:07<00:00, 62.63it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
(x_train_Dense2, x_test_Dense2, y_train_Dense2, y_test_Dense2 ) = train_test_split(data_Dense, labels_Dense,
	                                                                                                test_size=0.25, random_state=40)

In [ ]:
le_Dense.classes_                  # มีค่า ['Lungcancer', 'Normal', 'Tuberculosis']

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [ ]:
INIT_LR = 1e-1
BS32 = 32
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:400]:    #fix w & bias in layer 0-400
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS32),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS32,
	                                            epochs=30,  callbacks = [callback])

Epoch 1/30
10/10 [==============================] - 159s 14s/step - loss: 0.5526 - accuracy: 0.6078 - val_loss: 0.5262 - val_accuracy: 0.6106
Epoch 2/30
10/10 [==============================] - 139s 14s/step - loss: 0.3489 - accuracy: 0.8156 - val_loss: 0.5189 - val_accuracy: 0.6460
Epoch 3/30
10/10 [==============================] - 134s 14s/step - loss: 0.2790 - accuracy: 0.8497 - val_loss: 0.4708 - val_accuracy: 0.6726
Epoch 4/30
10/10 [==============================] - 135s 14s/step - loss: 0.2392 - accuracy: 0.8693 - val_loss: 0.5354 - val_accuracy: 0.6637
Epoch 5/30
10/10 [==============================] - 134s 14s/step - loss: 0.1892 - accuracy: 0.9183 - val_loss: 0.4685 - val_accuracy: 0.6726
Epoch 6/30
10/10 [==============================] - 135s 14s/step - loss: 0.1611 - accuracy: 0.9412 - val_loss: 0.4691 - val_accuracy: 0.6726


In [ ]:
pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_Dense2.argmax(axis=1),  
                                    target_names=le_Dense.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.74      0.40      0.52        35
      Normal       0.67      1.00      0.80        41
Tuberculosis       0.64      0.57      0.60        37

    accuracy                           0.67       113
   macro avg       0.68      0.66      0.64       113
weighted avg       0.68      0.67      0.65       113



In [ ]:
score = metrics.accuracy_score(y_test_Dense2.argmax(axis=1),	        #ค่า acc
                                             pred_Dense2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

Fold score (accuracy): 0.672566371681416


In [ ]:
pred = np.round(pred_Dense2, 3)

##32 softmax   --> Crop

0.67 // 35 min

*   BS = 32
*   fix weight & bias in layer 0-400
*   activation="sigmoid"
*   learning_rate=0.0001
*   epoch 30

In [ ]:
train_img_Dense = []
mywidth = 224 
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป

    img = Image.open(image_path).convert('RGB')     

    wpercent = (mywidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
    img = img.crop((0,0,mywidth,mywidth))

    data_Dense = np.array(img, dtype="float32") / 255.0
    train_img_Dense.append(data_Dense)

data_Dense = np.array(train_img_Dense)

100%|██████████| 451/451 [00:07<00:00, 63.16it/s] 


In [ ]:
labels_Dense = np.array(data_train3[['NameType']] )        
labels_Dense.shape = (len(labels_Dense),) 

le_Dense = LabelEncoder()                                                                     
labels_Dense = le_Dense.fit_transform(labels_Dense)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_Dense = to_categorical(labels_Dense, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
(x_train_Dense2, x_test_Dense2, y_train_Dense2, y_test_Dense2 ) = train_test_split(data_Dense, labels_Dense,
	                                                                                                test_size=0.25, random_state=40)

In [ ]:
le_Dense.classes_                  # มีค่า ['Lungcancer', 'Normal', 'Tuberculosis']

array(['Lungcancer', 'Normal', 'Tuberculosis'], dtype=object)

In [ ]:
INIT_LR = 1e-1
BS32 = 32
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model_Dense = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
len(base_model_Dense.layers)

707

In [ ]:
base_model_Dense.trainable = True
for layer in base_model_Dense.layers[:400]:    #fix w & bias in layer 0-400
    layer.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Dense.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model_Dense2 = tf.keras.models.Model(inputs=base_model_Dense.input, outputs=prediction_layer)

model_Dense2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
H_Dense2 = model_Dense2.fit( x=aug2.flow(x_train_Dense2, y_train_Dense2, batch_size=BS32),
	                                            validation_data=(x_test_Dense2, y_test_Dense2),
	                                            steps_per_epoch=len(x_train_Dense2) // BS32,
	                                            epochs=30,  callbacks = [callback])

Epoch 1/30
10/10 [==============================] - 158s 14s/step - loss: 0.4961 - accuracy: 0.6438 - val_loss: 0.6678 - val_accuracy: 0.4336
Epoch 2/30
10/10 [==============================] - 138s 14s/step - loss: 0.3734 - accuracy: 0.7647 - val_loss: 0.6802 - val_accuracy: 0.4425
Epoch 3/30
10/10 [==============================] - 139s 14s/step - loss: 0.2863 - accuracy: 0.8333 - val_loss: 0.7181 - val_accuracy: 0.4159
Epoch 4/30
10/10 [==============================] - 138s 14s/step - loss: 0.2369 - accuracy: 0.8725 - val_loss: 0.7649 - val_accuracy: 0.4425
Epoch 5/30
10/10 [==============================] - 138s 14s/step - loss: 0.2068 - accuracy: 0.8889 - val_loss: 0.7925 - val_accuracy: 0.4513
Epoch 6/30
10/10 [==============================] - 138s 15s/step - loss: 0.1872 - accuracy: 0.9216 - val_loss: 0.8142 - val_accuracy: 0.4867
Epoch 7/30
10/10 [==============================] - 139s 14s/step - loss: 0.1395 - accuracy: 0.9444 - val_loss: 0.7366 - val_accuracy: 0.5398
Epoch 

In [ ]:
pred_Dense2 = model_Dense2.predict(x_test_Dense2)
    
print(classification_report(y_test_Dense2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                    pred_Dense2.argmax(axis=1),  
                                    target_names=le_Dense.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.70      0.40      0.51        35
      Normal       0.67      1.00      0.80        41
Tuberculosis       0.66      0.57      0.61        37

    accuracy                           0.67       113
   macro avg       0.68      0.66      0.64       113
weighted avg       0.68      0.67      0.65       113



In [ ]:
score = metrics.accuracy_score(y_test_Dense2.argmax(axis=1),	        #ค่า acc
                                             pred_Dense2.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

Fold score (accuracy): 0.672566371681416


In [ ]:
pred = np.round(pred_Dense2, 3)